In [127]:
import pandas as pd
import glob
import json
import numpy as np
import ast
from tqdm import tqdm

# DBAASP parsing

In [128]:
def listofdicts_to_df(list_):
    activity_info = pd.DataFrame()
    if list_ != []:
        df_list = [pd.DataFrame.from_dict(dict_, orient='columns') for dict_ in list_]
        activity_info = pd.concat(df_list)
        if 'description' in activity_info.index:
            activity_info = activity_info.drop(['description'], axis=0)
    return activity_info

def flatten(list_):
    return [item for sublist in list_ for item in sublist]

def transfer_id(df, column):
    numbers = [len(x) for x in df[column]]
    id_list = flatten(list(map(lambda index, number: [index] * number, df.id.tolist(), numbers)))
    return id_list

def get_seq_len_less_than(df, seq_length):
    df_short = df[df['sequence'].apply(lambda x: len(str(x)) <= seq_length)]
    return df_short

def get_seq_len_more_than(df, seq_length):
    df_short = df[df['sequence'].apply(lambda x: len(str(x)) >= seq_length)]
    return df_short

def get_active_vs_bacterium(df, bacterium):
    df_short = df[df['targetSpecies'].str.contains(bacterium)]
    return df_short

def parse_str(s):
    try:
        return ast.literal_eval(str(s))
    except:
        return

def clean_concentration(df):
    replace_values = {'>' : '', ' ' : '', '<' : '', '=': ''} 
    clean_df = df.copy()
    clean_df = clean_df[~clean_df['concentration'].str.contains('-')]
    clean_df['clean_concentration'] = clean_df['concentration'].replace(replace_values, regex=True)
    clean_df['clean_concentration'] = [x.split('±')[0] for x in clean_df['clean_concentration']]
    clean_df['clean_concentration'] = [parse_str(x) for x in clean_df['clean_concentration']]
    clean_df['clean_concentration'] = [x if type(x) is not tuple else np.nan for x in clean_df['clean_concentration'] ]    
    clean_df = clean_df.dropna(subset=['clean_concentration'])    
    return clean_df

def filter_by_acitivity(df, activity_column, threshold, mode=['active', 'inactive']):
    if mode == 'active':
        df_short = df[df[activity_column] <= threshold]
    elif mode == 'inactive':
        df_short = df[df[activity_column] > threshold]
    return df_short

In [129]:
peptides = pd.DataFrame()
all_file_names = []
total = 0
unusual_amino_acids = 0
for j_file in glob.glob("yaml/*.yaml"):
    total += 1
    filename = j_file[j_file.rfind("/") + 1:]
    with open(j_file) as train_file:
        dict_train = json.load(train_file)
    if dict_train.get("unusualAminoAcids") != []:
        unusual_amino_acids += 1
        continue
    peptide = pd.DataFrame.from_dict(dict_train, orient='index')
    peptides = pd.concat([peptides, peptide], axis=1)
    all_file_names.append(filename)
peptides = peptides.T    
print ("Total number of sequences:", total)
print ("Total number of sequences with unusual AminoAcids:", unusual_amino_acids)
print ("Total number of valid sequence:", len(peptides))

Total number of sequences: 10324
Total number of sequences with unusual AminoAcids: 2357
Total number of valid sequence: 7967


In [130]:
# Drop entries without sequence
peptides['sequence'] = peptides['sequence'].str.strip()
peptides = peptides.dropna(subset=['sequence'])
peptides = peptides.reset_index().drop('index', axis=1)
len(peptides)

7873

In [131]:
# Get sequences longer than 4 and shorter than 25
before_len_filtering = len(peptides)
peptides = get_seq_len_less_than(peptides, 25)
peptides = get_seq_len_more_than(peptides, 4)
after_len_filtering = len(peptides)

print ("Peptides removed:", before_len_filtering - after_len_filtering)
print ("Total number of peptides within range:", after_len_filtering)

Peptides removed: 1784
Total number of peptides within range: 6089


In [132]:
peptides = peptides.loc[:, [
    'id',
    'name',
    'sequence',
    'nTerminus',
    'cTerminus',
    'targetActivities',
    'hemoliticCytotoxicActivities',
]]

In [133]:
sequence_info = peptides.drop(['targetActivities', 'hemoliticCytotoxicActivities'], axis=1)
# TODO: consider only standard c- and nTerminus
# cTerminal is usually amidated (-CONH2), nTerminal has a free amine group (-NH2))
sequence_info['cTerminus'] = [x['name'] if x is not None else np.nan for x in sequence_info['cTerminus']]
sequence_info['nTerminus'] = [x['name'] if x is not None else np.nan for x in sequence_info['nTerminus']]
len(sequence_info)

6089

# Activity analysis

In [134]:
activity_info = pd.concat([listofdicts_to_df(x) for x in tqdm(peptides['targetActivities'])])
activity_info['id'] = transfer_id(peptides, 'targetActivities')
activity_info = activity_info.reset_index().drop('index', axis=1)    

100%|██████████| 6089/6089 [01:27<00:00, 69.44it/s] 


This set of conditions is picked so that the activity can be compared between peptides. 

In [135]:
activity_info = activity_info[activity_info['activityMeasureValue'] == 'MIC']
activity_info = activity_info[activity_info['cfuGroup'] == '1E5 - 1E6'] 
allowed_media = ['MHB', 'MHA', 'LBB', 'LBA', 'TSB', 'TSA', 'CAMHB'] 
activity_info = activity_info[activity_info['medium'].isin(allowed_media)]

def clean_up_conditions(df):
    clean_df = df[df['ph'] == '']
    clean_df = clean_df[clean_df['saltType'] == '']
    clean_df = clean_df[clean_df['concentration'] != '']
    return clean_df

activity_info = clean_up_conditions(activity_info)
activity_info = clean_concentration(activity_info)

In [136]:
activity_info = activity_info.drop([
    'reference',
    'activity',
    'activityMeasureValue',
    'activityMeasureGroup', 
    'cfuGroup',
    'ph',
    'saltType',
    'ionicStrength',
    
], axis=1)

activity_info = activity_info.rename(columns={
    'concentration': 'mic_concentration',
    'clean_concentration': 'mic_clean_concentration',    
    'unit': 'mic_unit',
    'medium': 'mic_medium',
    'cfu': 'mic_cfu',
    'note': 'mic_note'
})

In [137]:
activity_info

,id,targetSpecies,mic_concentration,mic_unit,mic_medium,mic_cfu,mic_note,mic_clean_concentration
22,9893,Escherichia coli ATCC 25922,2,µM,MHB,5E5-1E6,,2.0
28,9893,Escherichia coli UB1005,8,µM,MHB,5E5-1E6,,8.0
29,9893,Salmonella typhimurium C77-31,8,µM,MHB,5E5-1E6,,8.0
30,9893,Staphylococcus aureus ATCC 29213,4,µM,MHB,5E5-1E6,,4.0
37,9893,Staphylococcus aureus ATCC 43300,4,µM,MHB,5E5-1E6,MRSA,4.0
...,...,...,...,...,...,...,...,...
54265,13575,Staphylococcus aureus ATCC 25923,64,µg/ml,MHB,5E5,,64.0
54266,8033,Staphylococcus aureus ATCC 25923,6.3,µM,LBB,1E6,,6.3
54268,8033,Enterococcus faecalis 981,3.1,µM,LBB,1E6,Clinical isolate,3.1
54269,8033,Nocardia asteroides 201118,3.1,µM,LBB,1E6,Clinical isolate,3.1


# Toxicity info

In [138]:
toxicity_info = pd.concat([listofdicts_to_df(x) for x in tqdm(peptides['hemoliticCytotoxicActivities'])])
toxicity_info['id'] = transfer_id(peptides, 'hemoliticCytotoxicActivities')
toxicity_info = toxicity_info.reset_index().drop('index', axis=1)    

100%|██████████| 6089/6089 [00:13<00:00, 467.25it/s]


In [139]:
toxicity_info = toxicity_info[toxicity_info['activityMeasureForLysisValue'] == '50% Hemolysis']
toxicity_info = clean_up_conditions(toxicity_info)
toxicity_info = clean_concentration(toxicity_info)

toxicity_info = toxicity_info.drop([
    'reference',
    'activity',
    'activityMeasureForLysisValue',
    'activityMeasureForLysisGroup', 
    'ph',
    'saltType',
    'ionicStrength'
], axis=1)

toxicity_info = toxicity_info.rename(columns={
    'concentration': 'hc50_concentration',
    'clean_concentration': 'hc50_clean_concentration',
    'unit': 'hc50_unit',
    'note': 'hc50_note'
})

In [140]:
activity_info = sequence_info.merge(activity_info, how='inner')
activity_info = activity_info.merge(toxicity_info, how='left')

In [141]:
activity_info

,id,name,sequence,nTerminus,cTerminus,targetSpecies,mic_concentration,mic_unit,mic_medium,mic_cfu,mic_note,mic_clean_concentration,targetCell,hc50_concentration,hc50_unit,hc50_note,hc50_clean_concentration
0,9893,W2,WRPGRWWRPGRW,NaN,AMD,Escherichia coli ATCC 25922,2,µM,MHB,5E5-1E6,,2.0,NaN,NaN,NaN,NaN,NaN
1,9893,W2,WRPGRWWRPGRW,NaN,AMD,Escherichia coli UB1005,8,µM,MHB,5E5-1E6,,8.0,NaN,NaN,NaN,NaN,NaN
2,9893,W2,WRPGRWWRPGRW,NaN,AMD,Salmonella typhimurium C77-31,8,µM,MHB,5E5-1E6,,8.0,NaN,NaN,NaN,NaN,NaN
3,9893,W2,WRPGRWWRPGRW,NaN,AMD,Staphylococcus aureus ATCC 29213,4,µM,MHB,5E5-1E6,,4.0,NaN,NaN,NaN,NaN,NaN
4,9893,W2,WRPGRWWRPGRW,NaN,AMD,Staphylococcus aureus ATCC 43300,4,µM,MHB,5E5-1E6,MRSA,4.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16234,13575,Oncocin [R14K],VDKPPYLPRPRPPKRIYNR,NaN,AMD,Staphylococcus aureus ATCC 25923,64,µg/ml,MHB,5E5,,64.0,NaN,NaN,NaN,NaN,NaN
16235,8033,Brevinin-1TP3,FLPGLIKVAVGVGSTILCKITKKC,NaN,NaN,Staphylococcus aureus ATCC 25923,6.3,µM,LBB,1E6,,6.3,NaN,NaN,NaN,NaN,NaN
16236,8033,Brevinin-1TP3,FLPGLIKVAVGVGSTILCKITKKC,NaN,NaN,Enterococcus faecalis 981,3.1,µM,LBB,1E6,Clinical isolate,3.1,NaN,NaN,NaN,NaN,NaN
16237,8033,Brevinin-1TP3,FLPGLIKVAVGVGSTILCKITKKC,NaN,NaN,Nocardia asteroides 201118,3.1,µM,LBB,1E6,Clinical isolate,3.1,NaN,NaN,NaN,NaN,NaN


## TODO: mess around with the thresholds, especially for toxicity. 
Alternatively choose non-toxic peptides as hc50_clean_concentration > mic_clean_concentration

In [149]:
ecoli = get_active_vs_bacterium(activity_info, 'ATCC 25922')
saureus = get_active_vs_bacterium(activity_info, 'ATCC 25923')

ecoli_active = filter_by_acitivity(ecoli, 'mic_clean_concentration', 32, 'active')
saureus_active = filter_by_acitivity(saureus, 'mic_clean_concentration', 32, 'active')

ecoli_inactive = filter_by_acitivity(ecoli, 'mic_clean_concentration', 32, 'inactive')
saureus_inactive = filter_by_acitivity(saureus, 'mic_clean_concentration', 32, 'inactive')

toxic = filter_by_acitivity(activity_info, 'hc50_clean_concentration', 32, 'active')
nontoxic =  filter_by_acitivity(activity_info, 'hc50_clean_concentration', 32, 'inactive')

In [150]:
print('E. coli active: ', len(ecoli_active))
print('E. coli inactive: ', len(ecoli_inactive))

print('S. aureus active: ', len(saureus_active))
print('S. aureus inactive: ', len(saureus_inactive))

print('Toxic: ', len(saureus_active))
print('Non-toxic: ', len(saureus_inactive))

E. coli active:  912
E. coli inactive:  397
S. aureus active:  592
S. aureus inactive:  289
Toxic:  592
Non-toxic:  289


# Save

In [144]:
def prepare_for_fasta(df):
    fasta_df = pd.DataFrame()
    fasta_df['header'] = '>' + 'DBAASP' + '_' + df['id'].map(str)
    fasta_df['sequence'] = df['sequence'] 
    return fasta_df

In [145]:
prepare_for_fasta(ecoli_active).to_csv('high_ecoli_activity.tsv', sep='\t', index=False, header=False)
prepare_for_fasta(ecoli_inactive).to_csv('low_ecoli_activity.tsv', sep='\t', index=False, header=False)

In [146]:
prepare_for_fasta(saureus_active).to_csv('high_saureus_activity.tsv', sep='\t', index=False, header=False)
prepare_for_fasta(saureus_inactive).to_csv('low_saureus_activity.tsv', sep='\t', index=False, header=False)

In [147]:
prepare_for_fasta(toxic).to_csv('toxic_pepides.tsv', sep='\t', index=False, header=False)
prepare_for_fasta(nontoxic).to_csv('nontoxic_peptides.tsv', sep='\t', index=False, header=False)

In [170]:
for file in [
    'high_ecoli_activity',
    'low_ecoli_activity',
    'high_saureus_activity',
    'low_saureus_activity',
    'toxic_pepides',
    'nontoxic_peptides',
]:
    ! awk '{{print ""$$1"\n"$$2}}' {file}.tsv > {file}.fasta